# nf_loto_platform AI データサイエンティスト Playground

このノートブックでは、`nf_loto_platform.agents` パッケージを利用して、
Curator → Planner → Forecaster → Reporter → Orchestrator の一連の流れを
ローカル環境で検証するためのサンプルコードをまとめる。


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import pandas as pd

# プロジェクトルートを sys.path に追加
ROOT = Path(__file__).resolve().parents[1]
sys.path.insert(0, str(ROOT / "src"))

from nf_loto_platform.agents import (
    AgentOrchestrator,
    CuratorAgent,
    EchoLLMClient,
    ForecasterAgent,
    PlannerAgent,
    ReporterAgent,
    TimeSeriesTaskSpec,
)

from nf_loto_platform.db.loto_repository import load_panel_by_loto

print("ROOT=", ROOT)


In [ ]:
# サンプルタスク定義
task = TimeSeriesTaskSpec(
    loto_kind="loto6",
    target_horizon=4,
    frequency="W",
    objective_metric="mae",
    allow_tsfm=True,
    allow_neuralforecast=True,
    allow_classical=False,
    max_training_time_minutes=10.0,
    mode="auto",
)
task


In [ ]:
# 依存コンポーネントを組み立て
llm = EchoLLMClient(max_chars=512)
curator = CuratorAgent(llm_client=llm)
planner = PlannerAgent()
forecaster = ForecasterAgent()
reporter = ReporterAgent(llm_client=llm)

orchestrator = AgentOrchestrator(
    curator=curator,
    planner=planner,
    forecaster=forecaster,
    reporter=reporter,
)
orchestrator


In [ ]:
# 実際の nf_loto_* テーブルが存在する環境でのみ実行してください。
# 注意: このセルは DB に接続し、実験を実行します。

TABLE_NAME = "nf_loto_panel"  # 実際の環境に合わせて変更
LOTO = "loto6"
UNIQUE_IDS = ["default"]  # 実際の unique_id に合わせて変更

outcome, report = orchestrator.run_full_cycle(
    task=task,
    table_name=TABLE_NAME,
    loto=LOTO,
    unique_ids=UNIQUE_IDS,
)

outcome, report.summary
